# Dataloading into Graph Neural Network / EdgeClassifer stage

**Goal**: Test the weighting and hard cut config of the data loading process

In [2]:
%load_ext autoreload
%autoreload 2

import os

import numpy as np
import pandas as pd
import yaml
import torch

from time import time as tt



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Inspect Data Sample

In [2]:
from gnn4itk_cf.stages.edge_classifier import InteractionGNN

In [34]:
input_dir = "/global/cfs/cdirs/m3443/data/GNN4ITK/CommonFrameworkExamples/Example_1_Dev/module_map/trainset"
sample = torch.load(os.path.join(input_dir, "event000000100.pyg"), map_location="cpu")

In [35]:
sample

Data(hit_id=[261380], x=[261380], y=[840561], z=[261380], r=[261380], phi=[261380], eta=[261380], region=[261380], cluster_x_1=[261380], cluster_y_1=[261380], cluster_z_1=[261380], cluster_x_2=[261380], cluster_y_2=[261380], cluster_z_2=[261380], norm_x=[261380], norm_y=[261380], norm_z_1=[261380], eta_angle_1=[261380], phi_angle_1=[261380], eta_angle_2=[261380], phi_angle_2=[261380], norm_z_2=[261380], track_edges=[2, 106333], particle_id=[106333], pt=[106333], radius=[106333], primary=[106333], nhits=[106333], pdgId=[106333], config=[1], event_id='000000100', edge_index=[2, 840561], truth_map=[106333])

## Test GNN dataloading

In [3]:
import yaml

In [4]:
with open("gnn_train.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [5]:
model = InteractionGNN(config)

Defining figures of merit


/global/u2/d/danieltm/ExaTrkX/GNN4ITkTeam/commonframework/gnn4itk_cf/stages/edge_classifier/models/interaction_gnn.py:89: UserWarning: Failed to define figures of merit, due to logger unavailable
  warnings.warn("Failed to define figures of merit, due to logger unavailable")


In [6]:
model.setup(stage="predict")

/global/u2/d/danieltm/ExaTrkX/GNN4ITkTeam/commonframework/gnn4itk_cf/utils/loading_utils.py:49: UserWarning: OPTIONAL feature [ghost] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/global/u2/d/danieltm/ExaTrkX/GNN4ITkTeam/commonframework/gnn4itk_cf/utils/loading_utils.py:49: UserWarning: OPTIONAL feature [shared] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/global/u2/d/danieltm/ExaTrkX/GNN4ITkTeam/commonframework/gnn4itk_cf/utils/loading_utils.py:49: UserWarning: OPTIONAL feature [module_id] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")


In [7]:
dataloaders = model.predict_dataloader()

In [11]:
dataloaders[0].dataset.__dict__

{'root': '/global/cfs/cdirs/m3443/data/GNN4ITK/CommonFrameworkExamples/Example_1_Dev/module_map',
 'transform': None,
 'pre_transform': None,
 'pre_filter': None,
 '_indices': None,
 'input_dir': '/global/cfs/cdirs/m3443/data/GNN4ITK/CommonFrameworkExamples/Example_1_Dev/module_map/',
 'data_name': 'trainset',
 'hparams': "aggregation":       sum
 "batchnorm":         False
 "data_split":        [80, 10, 10]
 "edge_cut":          0.5
 "emb_channels":      0
 "factor":            0.9
 "gpus":              1
 "hard_cuts":         {'pt': [1000, inf]}
 "hidden":            128
 "hidden_activation": SiLU
 "input_dir":         /global/cfs/cdirs/m3443/data/GNN4ITK/CommonFrameworkExamples/Example_1_Dev/module_map/
 "layernorm":         True
 "lr":                0.0001
 "max_epochs":        200
 "model":             InteractionGNN
 "n_graph_iters":     8
 "nb_edge_layer":     2
 "nb_node_layer":     2
 "node_features":     ['r', 'phi', 'z']
 "nodes":             1
 "output_activation": Tanh
 "

In [42]:
sample2 = model.trainset[0]

In [43]:
sample2

Data(hit_id=[261380], x=[261380], y=[840561], z=[261380], r=[261380], phi=[261380], eta=[261380], region=[261380], cluster_x_1=[261380], cluster_y_1=[261380], cluster_z_1=[261380], cluster_x_2=[261380], cluster_y_2=[261380], cluster_z_2=[261380], norm_x=[261380], norm_y=[261380], norm_z_1=[261380], eta_angle_1=[261380], phi_angle_1=[261380], eta_angle_2=[261380], phi_angle_2=[261380], norm_z_2=[261380], track_edges=[2, 106333], particle_id=[106333], pt=[106333], radius=[106333], primary=[106333], nhits=[106333], pdgId=[106333], config=[1], event_id='000000100', edge_index=[2, 840561], truth_map=[106333], weights=[840561])

In [44]:
(sample2.pt[sample2.truth_map >= 0] > 1000).sum()

tensor(13908)

In [45]:
sample2.weights.shape

torch.Size([840561])

In [46]:
(sample2.weights[~sample2.y] == 1.).all()

tensor(True)

In [47]:
(sample2.weights == 1.).sum()

tensor(784401)

In [48]:
(sample2.weights == 0.).sum()

tensor(43818)

In [49]:
(sample2.weights == 5.).sum()

tensor(12342)

## Testing Scored Graphs

In [3]:
input_dir = "/global/cfs/cdirs/m3443/data/GNN4ITK/CommonFrameworkExamples/Example_1_Dev/gnn/trainset"
sample = torch.load(os.path.join(input_dir, "event000000100.pyg"), map_location="cpu")

In [4]:
sample

Data(hit_id=[261380], x=[261380], y=[34557], z=[261380], r=[261380], phi=[261380], eta=[261380], region=[261380], cluster_x_1=[261380], cluster_y_1=[261380], cluster_z_1=[261380], cluster_x_2=[261380], cluster_y_2=[261380], cluster_z_2=[261380], norm_x=[261380], norm_y=[261380], norm_z_1=[261380], eta_angle_1=[261380], phi_angle_1=[261380], eta_angle_2=[261380], phi_angle_2=[261380], norm_z_2=[261380], track_edges=[2, 14598], particle_id=[14598], pt=[14598], radius=[14598], primary=[14598], nhits=[14598], pdgId=[14598], config=[2], event_id='000000100', edge_index=[2, 34557], truth_map=[14598], weights=[34557], scores=[34557])